In [3]:
from tkinter import *
from PIL import Image, ImageTk
from tkinter import ttk
import random
from datetime import datetime 
import time

# Initialize game variables
is_paused = False
total_health = 100
bullets = 20
score = 0
active_zombies = []
name = ""
current_gun="AK"
wave = 1
zombies_spawned_in_wave = 0
zombies_per_wave = 10
boss_health=200


x_pos = 0  # Starting x position
y_pos = 40  # Starting y position (adjust as needed)

def show_popupcommand():
    popup = Toplevel(root)
    popup.title("Settings")
    popup.geometry("1500x1500")
    notebook = ttk.Notebook(popup)
    tab1 = Frame(notebook)
    tab2 = Frame(notebook)
    notebook.pack(expand=1, fill='both')
    notebook.add(tab1, text='Info')
    notebook.add(tab2, text='Controls')

    # Control labels for the second tab
    controls = [
        "Up = W", "Left = A", "Down = S", "Right = D", 
        "Mouse Control = M", "Keys (WASD or Arrow Keys) Control = K",
        "Fire = Right Click or Space", "Start = B", "Pause = P","Assault Rifle - 1","Shotgun - 2","Sniper - 3"]
    
    for control in controls:
        Label(tab2, text=control).pack(anchor='w')

    # Info tab
    Label(tab1, text="Bullets Replenish Every Two Seconds").pack(anchor='w')
    Label(tab1, text="If Your Health Reaches 0 The Game Ends").pack(anchor='w')
    Label(tab1, text="").pack(anchor='w')
    Label(tab1, text="Assault Rifle - Reloads One Bullet Per Second - Does One Damage").pack(anchor='w')
    Label(tab1, text="Shotgun - Reloads One Bullet Every Two Second - Does Two Damage").pack(anchor='w')
    Label(tab1,text="").pack(anchor='w')

    # Zombie information with images
    zombies = [("zom1.png", "Zombie 1"), ("zom2.png", "Zombie 2"), ("zom3.png", "Zombie 3"), ("zom4.png", "Zombie 4")]
    for num, (img, label_text) in enumerate(zombies, start=1):
        image = Image.open(img)
        resized_image = image.resize((20, 40), Image.Resampling.LANCZOS)
        photo = ImageTk.PhotoImage(resized_image)

        frame = Frame(tab1)
        frame.pack(anchor='w')
        
        Label(frame, image=photo).pack(side=LEFT)
        Label(frame, image=photo).image = photo
        Label(frame, text=f"{label_text} Has {num} Health").pack(side=LEFT)


# Check if game is over
def check_game_over():
    if total_health <= 0:
        game_over()

def move_bullet(event=None):
    global bullets, score, current_gun, boss_health
    if bullets > 0:
        bullets -= 1
        update_bullets_display()

        bullet_image = Image.open("bullet.png")
        resized_bullet = bullet_image.resize((45, 10), Image.Resampling.LANCZOS)
        bullet_pic = ImageTk.PhotoImage(resized_bullet)
        bullet_label = Label(root, image=bullet_pic)
        bullet_label.image = bullet_pic

        gun_x = x_pos + image1_width
        gun_y = y_pos + image1_height // 2
        bullet_label.place(x=gun_x, y=gun_y)

        def animate_bullet():
            global score, boss_health
            nonlocal gun_x
            try:
                if not root or not root.winfo_exists():
                    bullet_label.destroy()
                    return

                gun_x += 30
                bullet_label.place(x=gun_x, y=gun_y)

                # Check collision with the boss
                if boss_label and boss_label.winfo_exists():
                    boss_x, boss_y = boss_label.winfo_x(), boss_label.winfo_y()
                    if (gun_x + 45 > boss_x and gun_x < boss_x + imagebz_width and
                        gun_y + 10 > boss_y and gun_y < boss_y + imagebz_height):
                        damage = {"AK": 1, "Shotgun": 2, "Sniper": 4}.get(current_gun, 1)
                        boss_health -= damage
                        boss_progress['value'] = boss_health
                        score += damage
                        scorel.config(text=f"Score = {score}")

                        if boss_health <= 0:
                            boss_label.destroy()
                            game_over()  # End game when boss is defeated

                        bullet_label.destroy()
                        return

                if gun_x < root.winfo_width():
                    root.after(35, animate_bullet)
                else:
                    bullet_label.destroy()
            except TclError:
                bullet_label.destroy()

        animate_bullet()

# Modify the fireball function to handle the boss attack
def fireball(x_start, y_start):
    fireball_img = Image.open("fireball.png").resize((50, 50), Image.LANCZOS)
    fireball_pic = ImageTk.PhotoImage(fireball_img)
    fireball_label = Label(root, image=fireball_pic)
    fireball_label.image = fireball_pic
    fireball_label.place(x=x_start, y=y_start)

    def animate_fireball():
        nonlocal x_start
        if is_paused:
            root.after(100, animate_fireball)
            return

        x_start -= 20
        fireball_label.place(x=x_start, y=y_start)

        # Check collision with the player
        if x_start < 100 and y_pos < y_start < y_pos + image1_height:
            global total_health
            total_health -= 10
            update_health_display()
            fireball_label.destroy()
            return

        if x_start > -50:
            root.after(50, animate_fireball)
        else:
            fireball_label.destroy()

    animate_fireball()

# Start the boss battle at wave 4
def game_loop():
    global zombies_spawned_in_wave, zombies_per_wave, wave

    if not is_paused:
        if zombies_spawned_in_wave >= zombies_per_wave and len(active_zombies) == 0:
            if wave == 4:
                boss()
            else:
                wave += 1
                zombies_spawned_in_wave = 0
                zombies_per_wave += 10

                popupz = Toplevel(root)
                popupz.geometry("1500x1500")
                Label(popupz, text=f"Wave {wave} started!", font=("times", 100)).pack(anchor='center')
                root.after(2000, popupz.destroy)

    root.after(1, game_loop)
    
def move_bullet(event=None):
    global bullets, score, current_gun, boss_health
    if bullets > 0:
        bullets -= 1
        update_bullets_display()

        bullet_image = Image.open("bullet.png")
        resized_bullet = bullet_image.resize((45, 10), Image.Resampling.LANCZOS)
        bullet_pic = ImageTk.PhotoImage(resized_bullet)
        bullet_label = Label(root, image=bullet_pic)
        bullet_label.image = bullet_pic

        gun_x = x_pos + image1_width
        gun_y = y_pos + image1_height // 2
        bullet_label.place(x=gun_x, y=gun_y)

        def animate_bullet():
            global score
            nonlocal gun_x 
            # Use try-except to handle potential errors
            try:
                if not root or not root.winfo_exists():
                    bullet_label.destroy()
                    return
                

                gun_x += 30
                bullet_label.place(x=gun_x, y=gun_y)
                
                # Check collision with zombies
                for zombie in active_zombies.copy():
                    if zombie['label'] and zombie['label'].winfo_exists() and bullet_label.winfo_exists():
                        if (bullet_label.winfo_x() + 45 > zombie['label'].winfo_x() and 
                            bullet_label.winfo_x() < zombie['label'].winfo_x() + 80 and
                            bullet_label.winfo_y() + 10 > zombie['label'].winfo_y() and 
                            bullet_label.winfo_y() < zombie['label'].winfo_y() + 100):

                            damage = {"AK": 1, "Shotgun": 2, "Sniper": 4}.get(current_gun, 1)
                            zombie['health'] -= damage
                            score += damage
                            scorel.config(text=f"Score = {score}")

                            if zombie['health'] <= 0:
                                zombie['label'].destroy()
                                active_zombies.remove(zombie)

                            bullet_label.destroy()
                            return

                if gun_x < root.winfo_width():
                    root.after(35, animate_bullet)
                else:
                    bullet_label.destroy()
            except TclError:
                # Handle the exception gracefully
                bullet_label.destroy()

        animate_bullet()
def move_bullet(event=None):
    global bullets, score, current_gun, boss_health
    if bullets > 0:
        bullets -= 1
        update_bullets_display()

        bullet_image = Image.open("bullet.png")
        resized_bullet = bullet_image.resize((45, 10), Image.Resampling.LANCZOS)
        bullet_pic = ImageTk.PhotoImage(resized_bullet)
        bullet_label = Label(root, image=bullet_pic)
        bullet_label.image = bullet_pic

        gun_x = x_pos + image1_width
        gun_y = y_pos + image1_height // 2
        bullet_label.place(x=gun_x, y=gun_y)

        def animate_bullet():
            global score
            nonlocal gun_x 
            # Use try-except to handle potential errors
            try:
                if not root or not root.winfo_exists():
                    bullet_label.destroy()
                    return
                

                gun_x += 30
                bullet_label.place(x=gun_x, y=gun_y)
                
                # Check collision with zombies
                for zombie in active_zombies.copy():
                    if zombie['label'] and zombie['label'].winfo_exists() and bullet_label.winfo_exists():
                        if (bullet_label.winfo_x() + 45 > zombie['label'].winfo_x() and 
                            bullet_label.winfo_x() < zombie['label'].winfo_x() + 80 and
                            bullet_label.winfo_y() + 10 > zombie['label'].winfo_y() and 
                            bullet_label.winfo_y() < zombie['label'].winfo_y() + 100):

                            damage = {"AK": 1, "Shotgun": 2, "Sniper": 4}.get(current_gun, 1)
                            zombie['health'] -= damage
                            score += damage
                            scorel.config(text=f"Score = {score}")

                            if zombie['health'] <= 0:
                                zombie['label'].destroy()
                                active_zombies.remove(zombie)

                            bullet_label.destroy()
                            return

                if gun_x < root.winfo_width():
                    root.after(35, animate_bullet)
                else:
                    bullet_label.destroy()
            except TclError:
                # Handle the exception gracefully
                bullet_label.destroy()

        animate_bullet()
def move_bullet(event=None):
    global bullets, score, current_gun, boss_health
    if bullets > 0:
        bullets -= 1
        update_bullets_display()

        bullet_image = Image.open("bullet.png")
        resized_bullet = bullet_image.resize((45, 10), Image.Resampling.LANCZOS)
        bullet_pic = ImageTk.PhotoImage(resized_bullet)
        bullet_label = Label(root, image=bullet_pic)
        bullet_label.image = bullet_pic

        gun_x = x_pos + image1_width
        gun_y = y_pos + image1_height // 2
        bullet_label.place(x=gun_x, y=gun_y)

        def animate_bullet():
            global score
            nonlocal gun_x 
            # Use try-except to handle potential errors
            try:
                if not root or not root.winfo_exists():
                    bullet_label.destroy()
                    return
                

                gun_x += 30
                bullet_label.place(x=gun_x, y=gun_y)
                
                # Check collision with zombies
                for zombie in active_zombies.copy():
                    if zombie['label'] and zombie['label'].winfo_exists() and bullet_label.winfo_exists():
                        if (bullet_label.winfo_x() + 45 > zombie['label'].winfo_x() and 
                            bullet_label.winfo_x() < zombie['label'].winfo_x() + 80 and
                            bullet_label.winfo_y() + 10 > zombie['label'].winfo_y() and 
                            bullet_label.winfo_y() < zombie['label'].winfo_y() + 100):

                            damage = {"AK": 1, "Shotgun": 2, "Sniper": 4}.get(current_gun, 1)
                            zombie['health'] -= damage
                            score += damage
                            scorel.config(text=f"Score = {score}")

                            if zombie['health'] <= 0:
                                zombie['label'].destroy()
                                active_zombies.remove(zombie)

                            bullet_label.destroy()
                            return

                if gun_x < root.winfo_width():
                    root.after(35, animate_bullet)
                else:
                    bullet_label.destroy()
            except TclError:
                # Handle the exception gracefully
                bullet_label.destroy()

        animate_bullet()
def move_bullet(event=None):
    global bullets, score, current_gun, boss_health
    if bullets > 0:
        bullets -= 1
        update_bullets_display()

        bullet_image = Image.open("bullet.png")
        resized_bullet = bullet_image.resize((45, 10), Image.Resampling.LANCZOS)
        bullet_pic = ImageTk.PhotoImage(resized_bullet)
        bullet_label = Label(root, image=bullet_pic)
        bullet_label.image = bullet_pic

        gun_x = x_pos + image1_width
        gun_y = y_pos + image1_height // 2
        bullet_label.place(x=gun_x, y=gun_y)

        def animate_bullet():
            global score
            nonlocal gun_x 
            # Use try-except to handle potential errors
            try:
                if not root or not root.winfo_exists():
                    bullet_label.destroy()
                    return
                

                gun_x += 30
                bullet_label.place(x=gun_x, y=gun_y)
                
                # Check collision with zombies
                for zombie in active_zombies.copy():
                    if zombie['label'] and zombie['label'].winfo_exists() and bullet_label.winfo_exists():
                        if (bullet_label.winfo_x() + 45 > zombie['label'].winfo_x() and 
                            bullet_label.winfo_x() < zombie['label'].winfo_x() + 80 and
                            bullet_label.winfo_y() + 10 > zombie['label'].winfo_y() and 
                            bullet_label.winfo_y() < zombie['label'].winfo_y() + 100):

                            damage = {"AK": 1, "Shotgun": 2, "Sniper": 4}.get(current_gun, 1)
                            zombie['health'] -= damage
                            score += damage
                            scorel.config(text=f"Score = {score}")

                            if zombie['health'] <= 0:
                                zombie['label'].destroy()
                                active_zombies.remove(zombie)

                            bullet_label.destroy()
                            return

                if gun_x < root.winfo_width():
                    root.after(35, animate_bullet)
                else:
                    bullet_label.destroy()
            except TclError:
                # Handle the exception gracefully
                bullet_label.destroy()

        animate_bullet()
# Create a zombie with increasing speed and health based on the current wave
def create_zombie():
    global wave
    x = root.winfo_width()
    y = random.randint(50, frame_height - 150)
    
    # Base health and speed for zombies
    base_health_list = [("zom1.png", 1), ("zom2.png", 2), ("zom3.png", 3), ("zom4.png", 4)]
    picran, base_health = random.choice(base_health_list)
    
    # Scale health and speed with the wave number
    zombie_health = base_health
    speed = random.randint(3, 8) + (wave)
    health = base_health + (wave-1)

    zombie_image = Image.open(picran)
    resized_zombie = zombie_image.resize((80, 100), Image.Resampling.LANCZOS)
    zombie_pic = ImageTk.PhotoImage(resized_zombie)
    zombie_label = Label(root, image=zombie_pic)
    zombie_label.image = zombie_pic
    zombie_label.place(x=x, y=y)

    active_zombies.append({'label': zombie_label, 'health': zombie_health})

    def animate_zombie():
        nonlocal x
        if is_paused:
            root.after(100, animate_zombie)
            return

        x -= speed
        if zombie_label.winfo_exists():
            zombie_label.place(x=x, y=y)

            if x < frame_width - 40:
                zombie_label.destroy()
                for zombie in active_zombies:
                    if zombie['label'] == zombie_label:
                        active_zombies.remove(zombie)
                        break

                global total_health
                total_health -= 10
                update_health_display()
                return

            if x > -80:
                root.after(50, animate_zombie)
            else:
                zombie_label.destroy()
                for zombie in active_zombies:
                    if zombie['label'] == zombie_label:
                        active_zombies.remove(zombie)
                        break

    animate_zombie()

def boss():
    global boss_health, total_health, is_paused, boss_label, imagebz_width, imagebz_height, boss_progress

    # Stop spawning and animating zombies
    global active_zombies, zombies_spawned_in_wave, zombies_per_wave
    is_paused = True
    for zombie in active_zombies:
        if zombie['label'].winfo_exists():
            zombie['label'].destroy()
    active_zombies.clear()
    zombies_spawned_in_wave = zombies_per_wave  # Stop new zombies from spawning

    popupz = Toplevel(root)
    popupz.geometry("1500x1500")
    Label(popupz, text="Boss Wave Started!", font=("times", 40)).pack(anchor=CENTER)
    root.after(2000, popupz.destroy)

    boss_frame = Frame(root, width=200, height=300)
    boss_frame.place(x=600, y=100)

    boss_labelh = Label(boss_frame, text="Boss Health", font=("Georgia", 15), fg="yellow")
    boss_labelh.pack(side=TOP)

    boss_progress = ttk.Progressbar(boss_frame, orient='horizontal', length=200, mode='determinate')
    boss_progress['maximum'] = boss_health
    boss_progress['value'] = boss_health
    boss_progress.pack(side=TOP)

    imagebz = Image.open("zboss.png").resize((200, 300), Image.LANCZOS)
    zboss = ImageTk.PhotoImage(imagebz)
    imagebz_width, imagebz_height = imagebz.size

    boss_label = Label(boss_frame, image=zboss)
    boss_label.image = zboss
    boss_label.place(x=0, y=0)

    def boss_attack():
        if not is_paused:
            fireball(600, 150)  # Adjust starting position
            root.after(2000, boss_attack)  # Boss shoots a fireball every 2 seconds

    is_paused = False  # Resume the game but only for the boss
    boss_attack()  # Start the boss's attack loop

    

# Spawn zombies at intervals
def spawn_zombies(event=None):
    global zombies_spawned_in_wave, wave, zombies_per_wave

    if zombies_spawned_in_wave < zombies_per_wave and len(active_zombies) < 6 and random.random() < 0.5 and not is_paused:
        create_zombie()
        zombies_spawned_in_wave += 1

    root.after(1000, spawn_zombies)

    # Check if the wave is completed
    if zombies_spawned_in_wave >= zombies_per_wave and len(active_zombies) == 0:
        wave += 1
        zombies_spawned_in_wave = 0
        zombies_per_wave += 10  # Increase the number of zombies for the next wave

        # Show a popup for the new wave
        popupz = Toplevel(root)
        popupz.geometry("1500x1500")
        Label(popupz, text=f"Wave {wave} started!", font=("times", 100)).pack(anchor='center')
        root.after(1000, popupz.destroy)


# Start the game
def start_popup(event=None):
    global popup1  
    popup1 = Toplevel(root)
    popup1.geometry("1500x1000")
    Label(popup1, text="Pixel Gun Game", font=("Georgia", 40), fg="steel blue").pack(anchor=CENTER)
    Label(popup1, text="").pack(anchor=CENTER)
    Label(popup1, text="Name:", font=("Georgia", 20), fg="white").pack(anchor=CENTER)
    Label(popup1, text="").pack(anchor=CENTER)

    name_entry = Entry(popup1, font=("Georgia", 30), fg="white")
    name_entry.pack(anchor=CENTER)

    Button(popup1, text="Start", font=("Georgia", 25), fg="steel blue", bg="black", command=lambda: start(name_entry.get())).pack(anchor=CENTER)


def update_health_display():
    health_progress['value'] = total_health  # Update the health progress bar value
    check_game_over()

def update_bullets_display():
    try:
        print(bullet_progress)  # Print the widget to check if it's valid
        bullet_progress['value'] = bullets
    except TclError as e:
        pass

# Game over logic
def game_over():
    leader = Toplevel(root)
    leader.geometry("1500x1000")
    global active_zombies, name, score, is_paused, wave

    for zombie in active_zombies:
        zombie['label'].destroy()
    active_zombies.clear()
    with open("leaderboard.txt", "a") as file:
        file.write(f"Name : {name}  Score : {score}  Wave : {wave}\n\n")

    with open("leaderboard.txt", "r") as file:
        content = file.read()

    Label(leader, text=content,font=("Times",40)).pack(anchor=CENTER)

    is_paused = True
    root.unbind("<space>")
    root.unbind("<Button-1>")
    root.unbind('<Up>')
    root.unbind('<Down>')
    root.unbind('<w>')
    root.unbind('<s>')
    root.unbind('<Left>')
    root.unbind('<Right>')
    root.unbind('<a>')
    root.unbind('<d>')
    root.unbind('1')
    root.unbind('2')
    root.unbind('3')
    

# Movement functions
def move_image(x, y):
    global x_pos, y_pos
    new_x_pos = max(0, min(x_pos + x, frame_width - image1_width))
    new_y_pos = max(0, min(y_pos + y, frame_height - image1_height))

    x_pos, y_pos = new_x_pos, new_y_pos
    label.place(x=x_pos, y=y_pos)

def move_up(event): move_image(0, -35) 
def move_down(event): move_image(0, 35) 
def move_left(event): move_image(-35, 0) 
def move_right(event): move_image(35, 0) 

def move_imagemouse(event):
    global x_pos, y_pos
    new_x_pos = event.x_root - image1_width // 2
    new_y_pos = event.y_root - image1_height // 2

    new_x_pos = max(0, min(new_x_pos, frame_width - image1_width))
    new_y_pos = max(0, min(new_y_pos, frame_height - image1_height))

    x_pos = new_x_pos
    y_pos = new_y_pos
    label.place(x=x_pos, y=y_pos)

# Control settings
def keys(event=None):
    label.place(x=0, y=40)
    root.unbind('<Motion>')
    root.bind('<Up>', move_up)
    root.bind('<Down>', move_down)
    root.bind('<w>', move_up)
    root.bind('<s>', move_down)
    root.bind('<Left>', move_left)
    root.bind('<Right>', move_right)
    root.bind('<a>', move_left)
    root.bind('<d>', move_right)

def mouse(event=None):
    label.place(x=0, y=40)
    root.bind('<Motion>',move_imagemouse)
    root.unbind('<Up>')
    root.unbind('<Down>')
    root.unbind('<w>')
    root.unbind('<s>',)
    root.unbind('<Left>')
    root.unbind('<Right>')
    root.unbind('<a>')
    root.unbind('<d>')
    
# Replenish bullets every second
def switch_gun_by_key(event=None):
    global current_gun, label, photo1, photo2, photo3
    key = event.keysym
    if key == "1":
        current_gun = "AK"
        label.config(image=photo1)
        label.image = photo1
    elif key == "2":
        current_gun = "Shotgun"
        label.config(image=photo2)
        label.image = photo2
    elif key == "3":
        current_gun = "Sniper"
        label.config(image=photo3)
        label.image = photo3
    
def replenish_bullets():
    global bullets, current_gun
    if not is_paused and bullets < 20:
        bullets += 1
        update_bullets_display()
    if current_gun == "AK":
        reload_time = 1000
    elif current_gun == "Shotgun":
        reload_time = 3000
    elif current_gun == "Sniper":
        reload_time = 5000  # 4-second reload for sniper
    root.after(reload_time, replenish_bullets)



# Toggle pause functionality
def toggle_pause(event=None):
    global is_paused
    is_paused = not is_paused

    if is_paused:
        pause_label = Label(root, text="Paused", font=("Georgia", 40), fg="white")
        pause_label.place(relx=0.5, rely=0.5, anchor='center')
        root.update()
        root.unbind("<space>")
        root.unbind("<Button-1>")
        root.unbind('<Up>')
        root.unbind('<Down>')
        root.unbind('<w>')
        root.unbind('<s>')
        root.unbind('<Left>')
        root.unbind('<Right>')
        root.unbind('<a>')
        root.unbind('<d>')
        root.unbind('1')
        root.unbind('2')
        root.unbind('3')
        show_popupcommand()
    else:
        for widget in root.winfo_children():
            if isinstance(widget, Label) and widget.cget("text") == "Paused":
                widget.destroy()
                break

        root.bind("<space>", move_bullet)
        root.bind("<Button-1>", move_bullet)
        root.bind('<Up>', move_up)
        root.bind('<Down>', move_down)
        root.bind('<w>', move_up)
        root.bind('<s>', move_down)
        root.bind('<Left>', move_left)
        root.bind('<Right>', move_right)
        root.bind('<a>', move_left)
        root.bind('<d>', move_right)
        root.bind('1', switch_gun_by_key)
        root.bind('2', switch_gun_by_key)
        root.bind('3', switch_gun_by_key)
        popup.destroy()
# Initialize main window
root = Tk()
root.title("Pixel Gun Game") 
root.geometry("1500x1500")

frame = Frame(root, width=500, height=1000)
frame.pack(side=LEFT)
frame_width = 500
frame_height = 1000

# Status frame for health, bullets, and score
status_frame = Frame(root)
status_frame.pack(side=TOP, fill=X)

style_health = ttk.Style()
style_health.configure("Health.TProgressbar", foreground="green", background="red")

style_bullets = ttk.Style()
style_bullets.configure("Bullets.TProgressbar", foreground="blue", background="grey")

health_label = Label(status_frame, text=f"Health", font=("Georgia", 30), fg="red")
health_label.pack(side=LEFT, padx=(10, 0))  
health_progress = ttk.Progressbar(status_frame, orient='horizontal', length=100, mode='determinate', style="Health.TProgressbar")
health_progress['maximum'] = total_health
health_progress.pack(side=LEFT, padx=(10, 0))

bullets_label = Label(status_frame, text=f"Bullets", font=("Georgia", 30), fg="blue")
bullets_label.pack(side=LEFT, padx=(10, 0))  
bullet_progress = ttk.Progressbar(status_frame, orient='horizontal', length=100, mode='determinate', style="Bullets.TProgressbar")
bullet_progress['maximum'] = 20  # Set the maximum to match the initial bullets count
bullet_progress.pack(side=LEFT, padx=(10, 0))


scorel = Label(status_frame, text=f"Score = {score}", font=("Georgia", 30))
scorel.pack(side=LEFT, padx=(10, 0))  

zombies = [("zom1.png", "Zombie 1"), ("zom2.png", "Zombie 2"), ("zom3.png", "Zombie 3"), ("zom4.png", "Zombie 4")]
for num, (img, label_text) in enumerate(zombies, start=1):
    imagez = Image.open(img)
    resized_imagez = imagez.resize((20, 40), Image.Resampling.LANCZOS)
    photoz = ImageTk.PhotoImage(resized_imagez)
        
    Label(root, image=photoz).pack(side=LEFT,anchor=S)
    Label(root, image=photoz).image = photoz
    Label(root, text=f"{label_text} Has {num} Health").pack(side=LEFT,anchor=S)

# Settings button
# Settings button
settings_image = Image.open("settings.png").resize((50, 50), Image.LANCZOS)
settings_pic = ImageTk.PhotoImage(settings_image)
settings_button = Button(root, image=settings_pic, command=show_popupcommand)
settings_button.image = settings_pic
settings_button.pack(side=RIGHT, padx=(10, 0))


imageb = Image.open("black.png").resize((20, 1000), Image.LANCZOS)
black = ImageTk.PhotoImage(imageb)
imageb_width, imageb_height = imageb.size

marker_x = frame_width - 50
marker_y = frame_height - 1000
black_marker = Label(frame, image=black)
black_marker.place(x=marker_x, y=marker_y)
black_marker.image = black  # Keep a reference to the image

# Update settings button position on resize
def update_settings_button_position(event):
    settings_button.place(x=root.winfo_width() - 60, y=10)

root.bind('<Configure>', update_settings_button_position)

# Load gun images
image1 = Image.open("ak.png").resize((250, 100), Image.Resampling.LANCZOS)
photo1 = ImageTk.PhotoImage(image1)
image1_width, image1_height = image1.size

image2 = Image.open("shotgun.png").resize((250, 150), Image.Resampling.LANCZOS)
photo2 = ImageTk.PhotoImage(image2)
image2_width, image2_height = image2.size

image3 = Image.open("sniper.png").resize((250, 75), Image.Resampling.LANCZOS)
photo3 = ImageTk.PhotoImage(image3)
image3_width, image3_height = image3.size

label = Label(frame, image=photo1)
label.image = photo1
label.place(x=0, y=40)
root.update()

# Start the game
def start(name_input):
    global popup1, name ,zombies_spawned_in_wave, wave
    zombies_spawned_in_wave = 0
    name = name_input
    popup1.destroy()
    root.unbind("<b>")
    root.bind("<space>", move_bullet)
    root.bind("<Button-1>", move_bullet)
    root.bind('<Up>', move_up)
    root.bind('<Down>', move_down)
    root.bind('<w>', move_up)
    root.bind('<s>', move_down)
    root.bind('<Left>', move_left)
    root.bind('<Right>', move_right)
    root.bind('<a>', move_left)
    root.bind('<d>', move_right)
    root.bind('<k>', keys)
    root.bind('<m>', mouse)
    root.bind('1', switch_gun_by_key)
    root.bind('2', switch_gun_by_key)
    root.bind('3', switch_gun_by_key)
    spawn_zombies()  
    replenish_bullets()
    game_loop()
    popupz1 = Toplevel(root)
    popupz1.geometry("1500x1500") 
    Label(popupz1, text=f"Wave {wave} started!", font=("times", 100)).pack(anchor='center')
    root.after(1000, popupz1.destroy)
root.bind('<b>', start_popup)
root.bind('<p>', toggle_pause)
root.mainloop()